In [22]:
import os
from openai import AzureOpenAI
import time
import requests
import json
import concurrent.futures
from dotenv import load_dotenv

# .env 파일에서 환경 변수를 로드합니다.
load_dotenv()

# 환경변수에서 Azure OpenAI 클라이언트 설정
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),  # API 키
    api_version=os.getenv("AZURE_OPENAI_API_VERSION"),  # API 버전
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")  # Azure OpenAI 엔드포인트
)
print("Azure OpenAI 클라이언트 설정 완료")

# Coinone API에서 가격 정보를 가져오는 함수
def get_crypto_price(crypto_symbol):
    url = "https://api.coinone.co.kr/ticker?currency=all"  # Coinone API URL
    response = requests.get(url)  # API 요청
    data = response.json()  # JSON 응답 파싱
    print(f"Coinone API 응답 데이터: {data}")
    if crypto_symbol.lower() in data:  # 요청한 암호화폐 심볼이 데이터에 있는지 확인
        return float(data[crypto_symbol.lower()]['last'])  # 마지막 가격 반환
    else:
        return None  # 심볼이 없으면 None 반환

# Alpha Vantage API에서 주식 가격을 가져오는 함수
def get_stock_price(stock_symbol):
    api_key = os.getenv("ALPHA_VANTAGE_API_KEY")
    url = f"https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol={stock_symbol}&apikey={api_key}"
    response = requests.get(url)
    data = response.json()
    print(f"Alpha Vantage API 응답 데이터: {data}")
    if "Global Quote" in data and "05. price" in data["Global Quote"]:
        return float(data["Global Quote"]["05. price"])
    else:
        print(f"Error fetching stock price: {data}")
        return None

# Assistant 생성
assistant = client.beta.assistants.create(    
    name="Financial Data Checker", # Assistant 이름
    instructions="You are a financial data checker. Use the get_crypto_price function for cryptocurrencies and get_stock_price function for stocks to provide current prices.",
    model="gpt-4o",  # 모델 설정
    tools=[
        {
            "type": "function",  # 도구 타입
            "function": {
                "name": "get_crypto_price",  # 함수 이름
                "description": "Get the current price of a cryptocurrency",  # 함수 설명
                "parameters": {
                    "type": "object",  # 파라미터 타입
                    "properties": {
                        "crypto_symbol": {
                            "type": "string",  # 파라미터 데이터 타입
                            "description": "The symbol of the cryptocurrency (e.g., BTC, ETH)"  # 파라미터 설명
                        }
                    },
                    "required": ["crypto_symbol"]  # 필수 파라미터
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "get_stock_price",
                "description": "Get the current price of a stock",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "stock_symbol": {
                            "type": "string",
                            "description": "The symbol of the stock (e.g., AAPL, GOOGL)"
                        }
                    },
                    "required": ["stock_symbol"]
                }
            }
        }]
)
print("Assistant 생성 완료")

# 대화 스레드 생성
thread = client.beta.threads.create(    
    messages=[
        {
            "role": "user",
            "content": "현재 디즈니 주식 가격과 리플 가격은 얼마인가요?"
        }
    ]
)
print("대화 스레드 생성 완료")

# Assistant 실행
run = client.beta.threads.runs.create(
    thread_id=thread.id,  # 스레드 ID
    assistant_id=assistant.id  # Assistant ID
)
print("Assistant 실행 시작")

# 도구 호출을 처리하는 함수
def process_tool_call(tool_call):
    print(f"도구 호출 처리 시작: {tool_call}")
    if tool_call.function.name == "get_crypto_price":
        arguments = json.loads(tool_call.function.arguments)
        crypto_symbol = arguments.get("crypto_symbol")
        price = get_crypto_price(crypto_symbol)
        print(f"암호화폐 가격 조회 결과: {price}")
        return {
            "tool_call_id": tool_call.id,
            "output": str(price)
        }
    elif tool_call.function.name == "get_stock_price":
        arguments = json.loads(tool_call.function.arguments)
        stock_symbol = arguments.get("stock_symbol")
        price = get_stock_price(stock_symbol)
        print(f"주식 가격 조회 결과: {price}")
        return {
            "tool_call_id": tool_call.id,
            "output": str(price)
        }

# 병렬로 도구 호출을 처리하고 결과를 수집하는 함수
def process_tool_calls_parallel(tool_calls):
    tool_outputs = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_tool_call, tool_call) for tool_call in tool_calls]
        for future in concurrent.futures.as_completed(futures):
            tool_outputs.append(future.result())
    print(f"도구 호출 결과: {tool_outputs}")
    return tool_outputs

# 실행 상태 조회 및 도구 출력 제출
while True:
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    print(f"Run status: {run.status}")
    if run.status in ['queued', 'in_progress']:
        time.sleep(2)
    elif run.status == 'requires_action':
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        tool_outputs = process_tool_calls_parallel(tool_calls)
        client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id,
            run_id=run.id,
            tool_outputs=tool_outputs
        )
    else:
        break

# 응답 메시지 조회 및 출력
messages = client.beta.threads.messages.list(thread_id=thread.id)  # 메시지 리스트 조회
for message in reversed(list(messages)):  # 메시지 역순으로 출력
    print(f"{message.role}: {message.content[0].text.value}")  # 메시지 내용 출력

Azure OpenAI 클라이언트 설정 완료
Assistant 생성 완료
대화 스레드 생성 완료
Assistant 실행 시작
Run status: queued
Run status: in_progress
Run status: requires_action
도구 호출 처리 시작: RequiredActionFunctionToolCall(id='call_jduBE5HGA2yMPCHk7xLpiTL9', function=Function(arguments='{"stock_symbol": "DIS"}', name='get_stock_price'), type='function')
도구 호출 처리 시작: RequiredActionFunctionToolCall(id='call_xHxRAY5JxlSyVud34ji7KHcf', function=Function(arguments='{"crypto_symbol": "XRP"}', name='get_crypto_price'), type='function')
Coinone API 응답 데이터: {'result': 'success', 'errorCode': '0', 'timestamp': '1737346514', 'orca': {'currency': 'orca', 'first': '6000.0', 'low': '5250.0', 'high': '6950.0', 'last': '5400.0', 'volume': '17507.04647138', 'yesterday_first': '5500.0', 'yesterday_low': '5500.0', 'yesterday_high': '6500.0', 'yesterday_last': '6000.0', 'yesterday_volume': '29511.63319802'}, 'clv': {'currency': 'clv', 'first': '74.5', 'low': '74.5', 'high': '78.5', 'last': '78.5', 'volume': '5711.13620973', 'yesterday_first':